In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_nodes, export_model_as_feature
from train import run_experiment

2022-01-12 01:33:55.601435: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load audience overlap edges for level 1

In [2]:
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=1)
audience_overlap_sites_NODES = create_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-12 01:33:57 notebooks.utils INFO     Loaded 3489 nodes with records level <= 1 and child size:16981


[('crooked.com', 'votesaveamerica.com'), ('crooked.com', 'art19.com'), ('crooked.com', 'promocodeportal.com'), ('crooked.com', 'mediamatters.org'), ('crooked.com', 'actblue.com')]


In [3]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 10161 Sample: ['celebatheists.com', 'congress.org', 'bangkokbiznews.com', 'greenvillegazette.com', 'maphub.net']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10161 entries, celebatheists.com to writingcommons.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   7465 non-null   float64
 1   daily_pageviews_per_visitor  7466 non-null   float64
 2   daily_time_on_site           5566 non-null   float64
 3   total_sites_linking_in       9861 non-null   float64
 4   bounce_rate                  5179 non-null   float64
dtypes: float64(5)
memory usage: 476.3+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10161 entries, celebatheists.com to writingcommons.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   10161 non-null  float64
 1   daily_pageviews_per_visitor  7466 non-null   float64
 2   daily_time_on_site           5566 non-null   float64
 3   total_sites_linking_in       10161 non-null  float64
 4   bounce_rate                  5179 non-null   float64
dtypes: float64(5)
memory usage: 476.3+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 10161, Edges: 17010

 Node types:
  default: [10161]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [17010]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [11]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [12]:
history = model.fit(train_gen, epochs=32, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/32
1626/1626 - 14s - loss: 0.6956 - binary_accuracy: 0.5264 - 14s/epoch - 9ms/step
Epoch 2/32
1626/1626 - 14s - loss: 0.6691 - binary_accuracy: 0.5415 - 14s/epoch - 8ms/step
Epoch 3/32
1626/1626 - 19s - loss: 0.6629 - binary_accuracy: 0.5418 - 19s/epoch - 11ms/step
Epoch 4/32
1626/1626 - 14s - loss: 0.6597 - binary_accuracy: 0.5378 - 14s/epoch - 9ms/step
Epoch 5/32
1626/1626 - 18s - loss: 0.6592 - binary_accuracy: 0.5393 - 18s/epoch - 11ms/step
Epoch 6/32
1626/1626 - 14s - loss: 0.6552 - binary_accuracy: 0.5424 - 14s/epoch - 9ms/step
Epoch 7/32
1626/1626 - 15s - loss: 0.6555 - binary_accuracy: 0.5425 - 15s/epoch - 9ms/step
Epoch 8/32
1626/1626 - 16s - loss: 0.6510 - binary_accuracy: 0.5470 - 16s/epoch - 10ms/step
Epoch 9/32
1626/1626 - 17s - loss: 0.6518 - binary_accuracy: 0.5493 - 17s/epoch - 10ms/step
Epoch 10/32
1626/1626 - 14s - loss: 0.6501 - binary_accuracy: 0.5488 - 14s/epoch - 9ms/step
Epoch 11/32
1626/1626 - 14s - loss: 0.6514 - binary_accuracy: 0.5479 - 14s/epoch - 9m

In [13]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

204/204 [==============================] - 0s 1ms/step
Sample: [8.546439289602858e-07, 1.6193119165442554e-13, 6.823146214696152e-11, 0.017228275537490845, 2.230987547591212e-06, 6.467853336245e-11, 0.036241620779037476, 2.8712513085338287e-05, 0.01550513505935669, 0.0033212602138519287] 128


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, 'attrib2vec_audience_overlap_level_1')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/attrib2vec_audience_overlap_level_1.json'

In [15]:
run_experiment(features="attrib2vec_audience_overlap_level_1")

+------+---------------------+---------------+--------------------+-------------------------------------+
| task | classification_mode | type_training | normalize_features |               features              |
+------+---------------------+---------------+--------------------+-------------------------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1 |
+------+---------------------+---------------+--------------------+-------------------------------------+


01-12 01:42:30 train        INFO     Start training...
01-12 01:42:30 train        INFO     Fold: 0
01-12 01:42:36 train        INFO     Fold: 1
01-12 01:42:40 train        INFO     Fold: 2
01-12 01:42:44 train        INFO     Fold: 3
01-12 01:42:48 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+--------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |               features              |      Macro-F1     |      Accuracy      |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+--------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1 | 34.21754849999626 | 54.249126891734576 | 14.551804423748546 | 0.6030267753201397 |
+------+---------------------+---------------+--------------------+-------------------------------------+-------------------+--------------------+--------------------+--------------------+
